### Introdução ao uso de LangChain

O Langchain permite trabalhar facilmente com diversos modelos.

**Modelos:**

- [Documentação LangChain](https://python.langchain.com/docs/integrations/llms/)

### Instalação

In [17]:
# Para uso no colab, necessário executar está celula para instalar as bibliotecas necessárias
!pip install -q transformers einops accelerate bitsandbytes

!pip install -q langchain
!pip install -q langchain-community
!pip install -q langchain-huggingface
!pip install -q langchainhub
!pip install -q langchain_chroma
!pip install -q langchain_openai

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-classic 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.
langchain-classic 1.0.0 requires langchain-text-splitters<2.0.0,>=1.0.0, but you have langchain-text-splitters 0.3.11 which is incompatible.
langchain-community 0.4 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.
langchain-huggingface 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.
langchain-chroma 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requi

In [1]:
import torch
import os
import getpass # Para informar o token do huggingFace

# Transformers do HuggingFace
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig, AwqConfig

# Integrando Langchain juntamente com o Hugging Face
from langchain_huggingface import HuggingFacePipeline

from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    PromptTemplate
)

from langchain_core.messages import SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from dotenv import load_dotenv

In [3]:
load_dotenv()
hf_token = os.getenv("HF_TOKEN")

### Carregando LLM via pipeline (a mesma da aula anterior)

**Utilizando Langchain juntamente com o HuggingFace**

Aqui iremos utilizar o mesmo código e modelo da aula anterior, porém, utilizando o Langchain para pré processar os textos, sem a necessidade de criar templates como fizemos nas aulas com HuggingFace.

In [ ]:
model_id = "microsoft/Phi-3-mini-4k-instruct"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_id, # Modelo que será baixado
    device_map = "cuda", # Deve ser carregada em uma GPU habilitada para CUDA
    torch_dtype = "auto",
    trust_remote_code = False,
    attn_implementation="eager" # Método de implementação para o mecanismo de atenção
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

A temperatura controla a aleatoriedade do processo de geração de texto.

Valores mais baixos tornam a saída do modelo mais determinística, enquanto que valores mais altos aumentam a "criatividade" do algoritimo (algo que também pode causar a alucinação).

In [ ]:
pipe = pipeline(
    model = model,
    tokenizer = tokenizer,
    task = "text-generation",
    temperature = 0.1,
    max_new_tokens = 500, # Será gerado no máximo 500 tokens
    do_sample = True, # Adiciona fator de aleatoriedade na geração dos textos
    repetition_penalty = 1.1, # Desencoraja o modelo a gerar textos ou frases muito repetitivas. 1 é padrão (nenhuma penalidade)
    return_full_text = False # Determina se deve ser retornado o texto completo, incluindo o prompt completo ou apenas o texto gerado
)

Device set to use cuda


In [ ]:
llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
output = llm.invoke(input="Qual é o sentido da vida?")
print(output)



### Answer:O significado ou propósito da vida tem sido objeto de debate filosófico, espiritual e científico por séculos. A resposta a essa pergunta pode variar muito dependendo das crenças pessoais, culturais e religiosas individuais. Alguns argumentam que cada indivíduo deve encontrar seu próprio significado na vida através do auto-desenvolvimento, contribuição para os outros e busca de felicidade pessoal. Outros sugerem que um senso coletivo de bem estar humano, compreensão mútua e preocupação com questões sociais importantes como justiça, igualdade e sustentabilidade ambiental são aspectos cruciais do significado da vida. Em última análise, o significado da vida muitas vezes depende dos valores e experiências únicas de cada pessoa.


### Outros modelos Open Source (Llama)

- [Meta-Llama-3-8B-Instruct](https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct)

Com a máquina utilizada para fazer o curso, não foi possível baixar e utilizar o modelo llama a partir do HuggingFace nem com a versão quantizada, iremos continuar utilizando o modelo *Phi-3*.

In [ ]:
# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_id = "hugging-quants/Meta-Llama-3.1-8B-Instruct-AWQ-INT4"

### Adequando o prompt

Os tokens especiais usados para interagir via prompt com o **Llama 3** são esses:

```py
  template = """
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
  {system_prompt}
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
  {user_prompt}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""
```

* `<|begin_of_text|>`: equivalente ao token BOS (Beginning of String), indicando o início de uma nova sequência de texto.

* `<|eot_id|>`: indica o fim de uma mensagem.

* `<|start_header_id|>{role}<|end_header_id|>`: esses tokens envolvem o papel de uma mensagem específica. Os papéis possíveis são: system, user e assistant.

* `<|end_of_text|>`: Isso é equivalente ao token EOS (End of String). Ao chegar nesse token, o Llama 3 deixará de gerar mais tokens.

Os tokens usados para interagir via prompt com o **Phi-3** são esses:

- `<|system|>` Indica o prompt do sistema.
- `<|user|>` Indica o prompt do usuário.
- `<|assistant|>` Faz o modelo se comportar como um assistente.
- `<|end|>` Indica o fim de uma mensagem



In [ ]:
template = """
<|system|>
  {system_prompt}<|end|>
<|user|>
  {user_prompt}<|end|>
<|assistant|>
"""

In [ ]:
system_prompt = "Você é um assistente e está respondendo perguntas gerais."
input = "Qual foi a primeira linguagem de programação?"

In [ ]:
# Forma interessante para adicionar variáveis com a função format
prompt_template = template.format(system_prompt = system_prompt, user_prompt = input)
prompt_template

'\n<|system|>\n  Você é um assistente e está respondendo perguntas gerais.<|end|>\n<|user|>\n  Qual foi a primeira linguagem de programação?<|end|>\n<|assistant|>\n'

In [ ]:
# Enviando o prompt_template para o modelo
output = llm.invoke(prompt_template)
print(output)

 A questão sobre qual foi "a" primeira linguagem de programação não tem uma resposta definitiva, pois o conceito de linguagem de programação evoluiu ao longo do tempo com diferentes sistemas computacionais. No entanto, muitos historiadores da informática consideram que as máquinas Jacquard para tecelagem desenvolvidas por Joseph Marie Jacquard em meados do século XIX podem ser vistas como precursoras das primeiras linguagens de programação devido à sua capacidade de executar instruções complexas através de bobinas de fios padrões (que poderiam ser pensadas como comandos). Mas se falarmos especificamente na linha direta dos softwares modernos, então Ada Lovelace frequentemente é reconhecida pela criação do primeiro programa algorítmico no final do século XIX quando trabalhou com Charles Babbage nas etapas iniciais do projeto Analytical Engine. O código escrito por ela era baseado em matemática e lógica, mas nunca foi implementado durante a vida dela ou imediatamente após seu trabalho. P

### Modelos de Chat

In [ ]:
from langchain_core.messages import (HumanMessage, SystemMessage)
from langchain_huggingface import ChatHuggingFace

#### Adequando o modelo com templates

Aqui estamos definindo como o modelo irá se comportar. Diferentemente dos exemplos onde precisavamos definir as tags de finalização dos textos e quem era o ator (role) manualmente, desta vez podemos definir com as classes *SystemMessage* e *HumanMessage*.

In [ ]:
msgs = [
    SystemMessage(content="Você é um assistente e está respondendo perguntas gerais."),
    HumanMessage(content="Explique para mim brevemente o que seria a vida.")
]

In [ ]:
chat_model = ChatHuggingFace(llm=llm) # Aqui é definido qual LLM será utilizada, no momento, estamos utilizando o Phi-3

Abaixo é o template do modelo.

In [ ]:
model_template = tokenizer.chat_template
model_template

"{% for message in messages %}{% if message['role'] == 'system' %}{{'<|system|>\n' + message['content'] + '<|end|>\n'}}{% elif message['role'] == 'user' %}{{'<|user|>\n' + message['content'] + '<|end|>\n'}}{% elif message['role'] == 'assistant' %}{{'<|assistant|>\n' + message['content'] + '<|end|>\n'}}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '<|assistant|>\n' }}{% else %}{{ eos_token }}{% endif %}"

Abaixo é o template preenchido com os prompts.

In [ ]:
chat_model._to_chat_prompt(msgs)

'<|system|>\nVocê é um assistente e está respondendo perguntas gerais.<|end|>\n<|user|>\nExplique para mim brevemente o que seria a vida.<|end|>\n<|assistant|>\n'

Enviando o template de chat para o modelo.

In [ ]:
resp = chat_model.invoke(msgs)
print(resp.content)

 A vida, em sua essência mais simples, pode ser entendida como uma série de processos biológicos complexos que permitem aos organismos viver, crescer, se reproduzir e adaptar-se ao seu ambiente. É caracterizada pela capacidade de manter funções vitais por meio da energia obtida do metabolismo, utilizando compostos orgânicos básicos como carbono, hidrogênio, oxigênio e nitrogênio. Essa natureza dinâmica implica interações com outros seres vivos e elementos não vivos (fatores abióticos), formando ecossistemas diversos onde cada espécie desempenha papéis específicos no equilíbrio ambiental. Em resumo, a vida representa as manifestações concretas dos sistemas químicos fundamentais na Terra, organizados hierarquicamente desde moléculas até comunidades ecológicas.


### Prompt Templates

Os modelos de prompt (promt templates) ajudam a traduzir a entrada e os parâmetros do usuário em instruções para um modelo de linguagem. Pode ser usado para orientar a resposta de um modelo, ajudando-o a entender o contexto e gerar uma saída relevante. Também facilita a criação de prompts de maneiras variáveis.

Existem alguns tipos diferentes de modelos de prompt:

#### String PromptTemplates

Esses modelos de prompt são usados para formatar uma única string e geralmente são usados para entradas mais simples.

Tipo de template utilizada para enviar uma string como parâmetro.

In [ ]:
prompt_template = PromptTemplate.from_template("Escreva um poema sobre {topic}")
prompt_template.invoke({"topic": "abacates"})

StringPromptValue(text='Escreva um poema sobre abacates')

In [ ]:
prompt_template

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Escreva um poema sobre {topic}')

#### ChatPromptTemplates

São usados para formatar uma lista de mensagens. Ou seja, uma lista de templates.

In [ ]:
prompt = ChatPromptTemplate.from_messages({
    ("system", "Você é um assistente e está respondendo perguntas gerais."),
    ("user", "<|user|>Explique-me em 1 parâgrafo o conceito de {topic}<|end|><|assistant|>") # Necessário adicionar o prompt template do Phi-3 dentro do user para evitar que o modelo alucine
})

# Define a variável topic ao chamar a função invoke
prompt.invoke({"topic": "IA"})

ChatPromptValue(messages=[SystemMessage(content='Você é um assistente e está respondendo perguntas gerais.', additional_kwargs={}, response_metadata={}), HumanMessage(content='<|user|>Explique-me em 1 parâgrafo o conceito de IA<|end|><|assistant|>', additional_kwargs={}, response_metadata={})])

Enviando o ChatPromptTemplate via modelo com a técnica de Chain (será explicada em breve)

In [ ]:
chain = prompt | llm
output = chain.invoke({"topic": "programação"})
print(output)

 A programação consiste na criação, desenvolvimento e manutenção de software através da escrita de código que instrui computadores a realizar tarefas específicas ou processos complexos. É uma disciplina técnica fundamental no mundo digital moderno, permitindo aos usuários automatizar funções repetitivas, resolver problemas matemáticos intrincados e criar interfaces interativas para aplicações web e móveis. Os programadores utilizam linguagens de programação como Python, Java, C++ entre outras, seguindo princípios estruturais (como lógica condicional, loops) e algorítmicos para projetar soluções funcionais e eficientes. Além disso, envolve entender sistemas operacionais, bancos de dados e hardware subjacentes ao sistema alvo do programa. Com cada evolução tecnológica, novas paradigmas emergem, como programação orientada a objetos, funcional, reativa e baseada em eventos, refletindo as necessidades dos aplicativos contemporâneos.


Adicionando mais de uma variável.

In [ ]:
system_prompt = "Você é um assistente e está respondendo perguntas gerais."
user_prompt = "Explique para mim brevemente o conceito de {topic}, de forma clara e objetiva. Escreva em no máximo {tamanho}." # variáveis topic e tamanho

prompt = PromptTemplate.from_template(template.format(system_prompt = system_prompt, user_prompt = user_prompt))
prompt.invoke({"topic": "IA", "tamanho": "1 parágrafo"}) # Definindo as variáveis dentro do template do Phi-3

StringPromptValue(text='\n<|system|>\n  Você é um assistente e está respondendo perguntas gerais.<|end|>\n<|user|>\n  Explique para mim brevemente o conceito de IA, de forma clara e objetiva. Escreva em no máximo 1 parágrafo.<|end|>\n<|assistant|>\n')

In [ ]:
prompt # Note que prompt tem dois input_variables: "tamanho" e "topic"

PromptTemplate(input_variables=['tamanho', 'topic'], input_types={}, partial_variables={}, template='\n<|system|>\n  Você é um assistente e está respondendo perguntas gerais.<|end|>\n<|user|>\n  Explique para mim brevemente o conceito de {topic}, de forma clara e objetiva. Escreva em no máximo {tamanho}.<|end|>\n<|assistant|>\n')

Enviando o prompt com duas variáveis para o modelo com a técnica de Chain.

A saida do **prompt** será usada como entrada para a **llm**

In [ ]:
chain = prompt | llm
output = chain.invoke({"topic": "IA", "tamanho": "1 parágrafo"})
print(output)

 A Inteligência Artificial (IA) refere-se à simulação da inteligência humana por computadores ou sistemas automatizados. Essa tecnologia visa criar programas que possam realizar tarefas tipicamente associadas a seres humanos, como reconhecimento visual, processamento linguístico e tomada de decisões complexas. Através do aprendizado automático, onde os algoritmos analisam grandes quantidades de dados para identificar padrões e fazer previsões, a IA pode melhorar continuamente sua performance ao longo do tempo sem intervenção direta dos usuários. O impacto dessa área tem sido significativo em diversos setores, incluindo saúde, finanças e transporte, oferecendo soluções mais rápidas, precisas e personalizadas.


### Chains

Basicamente, as chains **encadeiam componentes**, onde a saída de um se torna a entrada do próximo, criando uma sequência lógica de operações.

A criação de uma corrente (chain) é dada pelo simbolo pipe `|`, exemplo:

```py
    chain = prompt | llm | StrOutputParser()
```

A ideia de fazer conexão entre os componentes é a base da linguagem de expressão do Langchain, que é chamada de **LCEL** (explicada no readme).

In [ ]:
# prompt -> é a variável criada anteriormente
# llm -> é o algoritmo ou modelo que foi carregado no início do código com a classe HuggingFacePipeline
chain = prompt | llm

Para executar o algoritmo é necessário chamar a função `invoke`.

In [ ]:
resp = chain.invoke({"topic": "vida", "tamanho": "1 frase"})
resp

' A vida refere-se às características que distinguem os seres vivos dos não-viventes, incluindo crescimento, reprodução, resposta a estímulos e adaptação ao ambiente através da evolução.'

#### Output Parser

O LangChain possui um componente chamado Output Parser (que significa "analisador de saída", tradução livre), que é responsável por processar a saída de um modelo em um **formato mais acessível ou adequado** para o nosso objetivo.

Isso é muito útil quando você está usando LLMs para gerar qualquer forma de dados estruturados. Por exemplo, uma formatação em html, xml, etc.

Em outras palavras, as vezes um modelo contem a resposta em um atributo "content" `resposta.content`. Basicamente o *StrOutputParser* passaria apenas o conteudo da resposta e não um objeto content com a resposta dentro.

**Dica:** Por mais que alguns modelos já retornem a resposta como string é uma boa prática sempre colocar o Output Parser no final da cadeia (chain).



In [ ]:
# Aqui é realizado a concatenação do promt + llm + Output parser
chain_str = chain | StrOutputParser() # Aqui já temos a ligação do prompt com a llm na variável chain

# Isso é equivalente a:
# chain_str = prompt | llm | StrOutputParser()

In [ ]:
chain_str.invoke({"topic": "vida", "tamanho": "1 frase"})

' A vida refere-se às características que distinguem os seres vivos dos não-viventes, incluindo crescimento, reprodução, resposta a estímulos e adaptação ao ambiente através da evolução.'

### Funções customizadas com Runnables (rodam em tempo de execução)

In [ ]:
from langchain_core.runnables import RunnableLambda

count = RunnableLambda(lambda x: f"Palavras: {len(x.split())}\n")

Desta forma agora o retorno do modelo também virá com a quantidade de palavras presentes no retorno da LLM.

In [ ]:
chain = prompt | llm | StrOutputParser() | count
chain.invoke({"topic": "criptografia", "tamanho": "1 frase"})

'Palavras: 18\n'

### Streaming

É utilizado para melhorar a experiência do usuário.

As vezes as LLMs podem demorar um pouco para retornar as respostas, então para resolver isto, é possível mostrar cada token (palavra) a medida que é gerado.

Isso permite que o usuário veja o progresso enquanto aguarda pela resposta e não apenas uma tela em branco ou de carregamento.

In [ ]:
# A função stream busca cada um dos tokens e copia para a variável "chunk" e exibe os resultados
for chunk in chain_str.stream({"topic": "buracos negros", "tamanho": "1 parágrafo"}):
  print(chunk, end=" ") # end=" " para renderizar na mesma linha

 Os    buracos   negros  são    regiões  do      espaço-tempo  onde  a   força   gravitacional  é   tão  forte  que   nada,  nem  mesmo  a    luz,  pode    escapar   deles  uma  vez    cruzado  seu    horizonte  de    eventos.   Essa   singularidade   densa   resulta  da    colapso   gravitacional  final  de    estrelas   massivas  após  sua  morte  na     supernova.  A   teoria  dos    buracos   negros  foi    formulada  por  Albert   Einstein  como  parte  de  suas   equações  da   relatividade    geral,  mas  só   ganhou    aceitação   científica  com  as    observações      astronômicas    recentes,   incluindo   imagens    diretas   feitas  pelo  Event   Horizon     Telescope.   Eles     desempenham  um  papel  fundamental  nas   ciências      astrofísicas   devido   às   influências   significativas  sobre  os   ambientes    circundantes  e  ao    potencial  para   revelar   novas   físicas   além  das   teorias      atuais. 

### Acesso de modelos via Hugging Face Hub



In [2]:
# Chave de API do Hugging Face
# os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HF_TOKEN")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass() # Para colab

··········


Carregamento do modelo do hugging face, no caso, do Llama 3 (não funciona)

In [15]:
from langchain_huggingface import HuggingFaceEndpoint

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

llm_hub = HuggingFaceEndpoint(
    repo_id = model_id,
    temperature = 0.1,
    return_full_text = False,
    max_new_tokens = 1024,
    task="text-generation",
    huggingfacehub_api_token=os.environ["HUGGINGFACEHUB_API_TOKEN"]
)

In [16]:
resp = llm_hub.invoke("O que seria um buraco negro?")
print(resp)

ValueError: Model meta-llama/Meta-Llama-3-8B-Instruct is not supported for task text-generation and provider novita. Supported task: conversational.

### Acessando modelos da Open AI (Exemplo: ChatGPT, modelo proprietário)

Necessário criar uma chave de api na OpenAI:
- https://platform.openai.com/settings/organization/api-keys

Os créditos de teste gratuitos foram descontinuados, portanto, a integração não funcionará, exceto se pagar pelo uso.

In [18]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API key: ")

OpenAI API key: ··········


In [22]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

In [23]:
msgs = [
    ("system", "Você é um assistente prestativo que responde a perguntas gerais."),
    ("human", "O que é um buraco negro?"),
]
ai_msg = chatgpt.invoke(msgs)
ai_msg

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

Notebook do professor:

Com anotações e outros exemplos de integração
- https://colab.research.google.com/drive/1GAmZyTCZhuHjKo2Fv8wBahfip0csMp52#scrollTo=NoVql31sGZ0h